## Local Inference on GPU 
Model page: https://huggingface.co/laion/CLIP-convnext_large_d_320.laion2B-s29B-b131K-ft-soup

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/laion/CLIP-convnext_large_d_320.laion2B-s29B-b131K-ft-soup)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [9]:
# ==========================================
# AI IMAGE DETECTION CON GRADIO SU KAGGLE
# ==========================================

# 1. INSTALLAZIONE DELLE DIPENDENZE
import subprocess
import sys

def install_packages():
    """Installa i pacchetti necessari su Kaggle"""
    packages = [
        'gradio',
        'open-clip-torch',
        'timm',  # Richiesto da open-clip
        'ftfy',  # Per il text processing
        'regex',  # Per il tokenizer
        'plotly',  # Per grafici interattivi
        'kaleido'  # Per esportare grafici plotly
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✓ {package} installato con successo")
        except subprocess.CalledProcessError:
            print(f"✗ Errore nell'installazione di {package}")

# Esegui installazione
install_packages()

# 2. IMPORT E CONFIGURAZIONE
import open_clip
import torch
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import gradio as gr
from pathlib import Path
import json
import warnings
import io
import base64
import time
warnings.filterwarnings('ignore')

# Configurazione dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo utilizzato: {device}")

# 3. CARICAMENTO DEL MODELLO
def load_clip_model():
    """Carica il modello CLIP ottimizzato per Kaggle"""
    try:
        model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
            'hf-hub:laion/CLIP-convnext_large_d_320.laion2B-s29B-b131K-ft-soup'
        )
        tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-convnext_large_d_320.laion2B-s29B-b131K-ft-soup')
        
        model = model.to(device)
        model.eval()
        
        print("✓ Modello CLIP caricato con successo")
        return model, preprocess_val, tokenizer
    except Exception as e:
        print(f"✗ Errore nel caricamento del modello: {e}")
        print("Tentativo con modello alternativo...")
        try:
            model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
                'ViT-B-32', pretrained='openai'
            )
            tokenizer = open_clip.get_tokenizer('ViT-B-32')
            model = model.to(device)
            model.eval()
            print("✓ Modello alternativo caricato con successo")
            return model, preprocess_val, tokenizer
        except Exception as e2:
            print(f"✗ Errore anche con modello alternativo: {e2}")
            return None, None, None

# Carica il modello
model, preprocess_val, tokenizer = load_clip_model()

# 4. CLASSE PER AI DETECTION
class AIImageDetector:
    def __init__(self, model, preprocess_val, tokenizer, device):
        self.model = model
        self.preprocess_val = preprocess_val
        self.tokenizer = tokenizer
        self.device = device
        
        # Prompt ottimizzati per diversi scenari
        self.detection_prompts = {
            'real_photo': [
                "a natural photograph with authentic lighting and realistic textures",
                "a real photo taken with a camera showing natural imperfections",
                "an authentic photograph with natural grain and lighting",
                "a genuine photo with realistic shadows and highlights",
                "a natural image with organic textures and realistic details"
            ],
            'ai_generated': [
                "an AI-generated image with synthetic artifacts and artificial patterns",
                "a digitally generated image created by artificial intelligence",
                "a synthetic image with overly smooth textures and perfect details",
                "an artificial image with computer-generated characteristics",
                "a machine-generated image with typical AI artifacts"
            ],
            'stable_diffusion': [
                "image generated by Stable Diffusion with characteristic noise patterns",
                "AI artwork with typical diffusion model artifacts and smooth textures",
                "Stable Diffusion generated image with distinctive rendering style"
            ],
            'midjourney': [
                "Midjourney generated image with distinctive artistic style",
                "AI art with Midjourney's characteristic color palette and composition",
                "Midjourney artwork with typical stylistic elements"
            ],
            'dalle': [
                "DALL-E generated image with typical OpenAI model characteristics",
                "AI image with DALL-E style rendering and texture patterns",
                "DALL-E artwork with characteristic generation artifacts"
            ]
        }
    
    def detect_single_image(self, image_input, detailed=True):
        """
        Rileva se una singola immagine è generata da AI
        """
        try:
            # Gestisce input diversi (path, PIL Image, numpy array)
            if isinstance(image_input, str):
                image = Image.open(image_input).convert('RGB')
            elif isinstance(image_input, np.ndarray):
                image = Image.fromarray(image_input).convert('RGB')
            else:
                image = image_input.convert('RGB')
            
            image_tensor = self.preprocess_val(image).unsqueeze(0).to(self.device)
            
            if detailed:
                return self._detailed_detection(image_tensor)
            else:
                return self._simple_detection(image_tensor)
                
        except Exception as e:
            return {'error': str(e)}
    
    def _simple_detection(self, image_tensor):
        """Rilevamento semplice e veloce"""
        prompts = [
            "a natural photograph with authentic lighting",
            "an AI-generated image with synthetic artifacts"
        ]
        
        text_tokens = self.tokenizer(prompts).to(self.device)
        
        with torch.no_grad():
            image_features = self.model.encode_image(image_tensor)
            text_features = self.model.encode_text(text_tokens)
            
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            
            similarities = (image_features @ text_features.T).softmax(dim=-1)[0]
        
        ai_probability = float(similarities[1])
        
        return {
            'ai_probability': ai_probability,
            'classification': 'AI-generated' if ai_probability > 0.5 else 'Real',
            'confidence': float(abs(similarities[1] - similarities[0]))
        }
    
    def _detailed_detection(self, image_tensor):
        """Rilevamento dettagliato con analisi per tipo di generatore"""
        all_prompts = []
        prompt_labels = []
        
        for category, prompts in self.detection_prompts.items():
            all_prompts.extend(prompts)
            prompt_labels.extend([category] * len(prompts))
        
        text_tokens = self.tokenizer(all_prompts).to(self.device)
        
        with torch.no_grad():
            image_features = self.model.encode_image(image_tensor)
            text_features = self.model.encode_text(text_tokens)
            
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            
            similarities = (image_features @ text_features.T).softmax(dim=-1)[0]
        
        # Raggruppa i risultati per categoria
        category_scores = {}
        for i, (score, label) in enumerate(zip(similarities, prompt_labels)):
            if label not in category_scores:
                category_scores[label] = []
            category_scores[label].append(float(score))
        
        # Calcola medie per categoria
        category_averages = {k: np.mean(v) for k, v in category_scores.items()}
        
        # Determina la classificazione
        max_category = max(category_averages, key=category_averages.get)
        is_ai = max_category != 'real_photo'
        
        return {
            'is_ai_generated': is_ai,
            'most_likely_source': max_category,
            'category_scores': category_averages,
            'confidence': float(category_averages[max_category]),
            'ai_probability': 1 - category_averages.get('real_photo', 0),
            'suspected_generator': max_category if is_ai else None
        }
    
    def batch_detect(self, image_paths, detailed=False):
        """
        Analizza multiple immagini in batch
        """
        results = []
        
        for i, image_path in enumerate(image_paths):
            print(f"Analizzando immagine {i+1}/{len(image_paths)}: {image_path}")
            
            result = self.detect_single_image(image_path, detailed=detailed)
            result['image_path'] = str(image_path)
            results.append(result)
        
        return results

# 5. FUNZIONI DI VISUALIZZAZIONE CON PLOTLY
def create_interactive_charts(results):
    """
    Crea grafici interattivi con Plotly
    """
    if isinstance(results, dict):
        results = [results]
    
    # Converte in DataFrame se necessario
    if not isinstance(results, pd.DataFrame):
        df = pd.DataFrame(results)
    else:
        df = results
    
    # Grafico a torta per classificazione
    classification_counts = df['classification'].value_counts() if 'classification' in df.columns else pd.Series(['Real', 'AI-generated'], index=[0, 1])
    
    fig_pie = px.pie(
        values=classification_counts.values,
        names=classification_counts.index,
        title="Distribuzione Classificazioni",
        color_discrete_map={'Real': 'green', 'AI-generated': 'red'}
    )
    
    # Grafico a barre per probabilità AI
    if len(df) > 1:
        fig_bar = px.histogram(
            df, 
            x='ai_probability', 
            title='Distribuzione Probabilità AI',
            nbins=20,
            labels={'ai_probability': 'Probabilità AI', 'count': 'Frequenza'}
        )
    else:
        fig_bar = go.Figure()
        fig_bar.add_trace(go.Bar(x=['AI Probability'], y=[df['ai_probability'].iloc[0]]))
        fig_bar.update_layout(title='Probabilità AI')
    
    return fig_pie, fig_bar

def create_detailed_analysis_chart(result):
    """
    Crea un grafico dettagliato per singola immagine
    """
    if 'category_scores' in result:
        categories = list(result['category_scores'].keys())
        scores = list(result['category_scores'].values())
        
        fig = px.bar(
            x=categories,
            y=scores,
            title="Punteggi per Categoria",
            labels={'x': 'Categoria', 'y': 'Punteggio'},
            color=scores,
            color_continuous_scale='RdYlBu_r'
        )
        
        fig.update_layout(showlegend=False)
        return fig
    
    return None

# 6. INTERFACCIA GRADIO
def create_gradio_interface():
    """
    Crea l'interfaccia Gradio per l'AI detection
    """
    if model is None:
        return gr.Interface(
            fn=lambda x: "Errore: Modello non caricato",
            inputs=gr.Image(type="pil"),
            outputs=gr.Textbox(),
            title="AI Image Detection - Errore"
        )
    
    detector = AIImageDetector(model, preprocess_val, tokenizer, device)
    
    def analyze_image(image, analysis_type):
        """
        Funzione principale per analizzare un'immagine
        """
        if image is None:
            return "Carica un'immagine per iniziare l'analisi", None, None, None
        
        start_time = time.time()
        detailed = analysis_type == "Dettagliata"
        
        result = detector.detect_single_image(image, detailed=detailed)
        
        if 'error' in result:
            return f"Errore nell'analisi: {result['error']}", None, None, None
        
        analysis_time = time.time() - start_time
        
        # Crea il testo di output
        output_text = f"""
## 🔍 Risultati dell'Analisi

**Classificazione:** {'🤖 AI-Generated' if result.get('is_ai_generated', result.get('classification') == 'AI-generated') else '📷 Real Photo'}

**Probabilità AI:** {result['ai_probability']:.1%}

**Confidence:** {result['confidence']:.1%}

**Tempo di analisi:** {analysis_time:.2f} secondi

---
"""
        
        if detailed and 'category_scores' in result:
            output_text += "\n### 📊 Analisi Dettagliata per Categoria:\n"
            for category, score in result['category_scores'].items():
                emoji = "🎨" if "ai" in category or category in ["stable_diffusion", "midjourney", "dalle"] else "📷"
                output_text += f"**{emoji} {category.replace('_', ' ').title()}:** {score:.1%}\n"
            
            if result['suspected_generator'] and result['suspected_generator'] != 'real_photo':
                output_text += f"\n**🎯 Generatore Sospetto:** {result['suspected_generator'].replace('_', ' ').title()}"
        
        # Crea grafici
        fig_pie, fig_bar = create_interactive_charts(result)
        detailed_chart = create_detailed_analysis_chart(result) if detailed else None
        
        return output_text, fig_pie, fig_bar, detailed_chart
    
    def analyze_batch(files, analysis_type):
        """
        Analizza multiple immagini
        """
        if not files:
            return "Carica almeno un'immagine", None, None, None
        
        detailed = analysis_type == "Dettagliata"
        results = []
        
        for file in files:
            try:
                image = Image.open(file.name)
                result = detector.detect_single_image(image, detailed=detailed)
                result['filename'] = file.name
                results.append(result)
            except Exception as e:
                results.append({'filename': file.name, 'error': str(e)})
        
        # Crea report
        total_images = len(results)
        ai_images = sum(1 for r in results if r.get('is_ai_generated', False) or r.get('classification') == 'AI-generated')
        real_images = total_images - ai_images
        
        output_text = f"""
## 📊 Analisi Batch Completata

**Totale immagini:** {total_images}
**Immagini AI:** {ai_images} ({ai_images/total_images:.1%})
**Immagini reali:** {real_images} ({real_images/total_images:.1%})

---

### 📋 Risultati per immagine:
"""
        
        for result in results:
            if 'error' in result:
                output_text += f"❌ **{result['filename']}:** Errore - {result['error']}\n"
            else:
                classification = '🤖 AI' if result.get('is_ai_generated', result.get('classification') == 'AI-generated') else '📷 Real'
                output_text += f"{classification} **{result['filename']}:** {result['ai_probability']:.1%} probabilità AI\n"
        
        # Crea grafici per il batch
        df = pd.DataFrame([r for r in results if 'error' not in r])
        if len(df) > 0:
            fig_pie, fig_bar = create_interactive_charts(df)
            return output_text, fig_pie, fig_bar, None
        else:
            return output_text, None, None, None
    
    # Interfaccia per singola immagine
    with gr.Blocks(title="AI Image Detection", theme=gr.themes.Soft()) as single_interface:
        gr.Markdown("""
        # 🔍 AI Image Detection
        ### Rileva se un'immagine è generata artificialmente o è una foto reale
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                image_input = gr.Image(
                    type="pil",
                    label="Carica un'immagine",
                    height=400
                )
                
                analysis_type = gr.Radio(
                    choices=["Semplice", "Dettagliata"],
                    value="Dettagliata",
                    label="Tipo di analisi"
                )
                
                analyze_btn = gr.Button("🔍 Analizza Immagine", variant="primary")
                
                gr.Markdown("""
                ### 📖 Come funziona:
                - **Semplice**: Veloce, determina solo se è AI o reale
                - **Dettagliata**: Identifica il possibile generatore AI (Stable Diffusion, Midjourney, DALL-E)
                """)
            
            with gr.Column(scale=2):
                output_text = gr.Markdown(label="Risultati")
                
                with gr.Row():
                    pie_chart = gr.Plot(label="Classificazione")
                    bar_chart = gr.Plot(label="Probabilità")
                
                detailed_chart = gr.Plot(label="Analisi Dettagliata", visible=False)
        print(output_text)
        analyze_btn.click(
            fn=analyze_image,
            inputs=[image_input, analysis_type],
            outputs=[output_text, pie_chart, bar_chart, detailed_chart]
        )
        
        # Mostra grafico dettagliato solo per analisi dettagliata
        analysis_type.change(
            fn=lambda x: gr.Plot(visible=(x == "Dettagliata")),
            inputs=analysis_type,
            outputs=detailed_chart
        )
    
    # Interfaccia per batch
    with gr.Blocks(title="AI Image Detection - Batch", theme=gr.themes.Soft()) as batch_interface:
        gr.Markdown("""
        # 📊 AI Image Detection - Batch Analysis
        ### Analizza multiple immagini contemporaneamente
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                files_input = gr.Files(
                    file_types=["image"],
                    label="Carica immagini multiple"
                )
                
                batch_analysis_type = gr.Radio(
                    choices=["Semplice", "Dettagliata"],
                    value="Semplice",
                    label="Tipo di analisi"
                )
                
                batch_analyze_btn = gr.Button("📊 Analizza Batch", variant="primary")
            
            with gr.Column(scale=2):
                batch_output = gr.Markdown(label="Risultati Batch")
                
                with gr.Row():
                    batch_pie = gr.Plot(label="Distribuzione")
                    batch_bar = gr.Plot(label="Probabilità")
        print(batch_output, batch_bar, batch_pie)

        batch_analyze_btn.click(
            fn=analyze_batch,
            inputs=[files_input, batch_analysis_type],
            outputs=[batch_output, batch_pie, batch_bar] 
        )
    
    # Combina le interfacce
    interface = gr.TabbedInterface(
        [single_interface, batch_interface],
        ["🔍 Analisi Singola", "📊 Analisi Batch"],
        title="AI Image Detection"
    )
    
    return interface

# 7. FUNZIONI UTILITY
def save_results_to_csv(results, filename="/kaggle/working/ai_detection_results.csv"):
    """
    Salva i risultati in CSV
    """
    if isinstance(results, dict):
        results = [results]
    
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Risultati salvati in {filename}")
    return df

def create_analysis_report(results, report_path="/kaggle/working/analysis_report.json"):
    """
    Crea un report dettagliato dell'analisi
    """
    if isinstance(results, dict):
        results = [results]
    
    df = pd.DataFrame(results)
    
    report = {
        'total_images': len(df),
        'ai_generated_count': int(df['is_ai_generated'].sum()) if 'is_ai_generated' in df.columns else 0,
        'real_images_count': int((~df['is_ai_generated']).sum()) if 'is_ai_generated' in df.columns else 0,
        'average_ai_probability': float(df['ai_probability'].mean()) if 'ai_probability' in df.columns else 0,
        'average_confidence': float(df['confidence'].mean()) if 'confidence' in df.columns else 0,
        'analysis_timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"Report salvato in {report_path}")
    return report

# 8. AVVIO DELL'APPLICAZIONE
def launch_app():
    """
    Avvia l'applicazione Gradio
    """
    print("🚀 Avvio dell'interfaccia AI Image Detection...")
    
    interface = create_gradio_interface()
    
    # Configurazione per Kaggle
    interface.launch(
        server_name="0.0.0.0",  # Permette accesso esterno
        server_port=7860,       # Porta standard
        share=True,             # Crea link pubblico
        debug=False,
        show_error=True,
        quiet=False
    )

# 9. ESECUZIONE PRINCIPALE
if __name__ == "__main__":
    print("=== AI IMAGE DETECTION CON GRADIO ===")
    
    if model is not None:
        print("✅ Modello caricato con successo!")
        print("🌐 Avvio interfaccia web...")
        launch_app()
    else:
        print("❌ Errore nel caricamento del modello")
        print("Verifica le dipendenze e riprova")
        
        # Interfaccia di errore
        error_interface = gr.Interface(
            fn=lambda: "❌ Modello non disponibile. Verifica l'installazione delle dipendenze.",
            inputs=gr.Textbox(placeholder="Modello non caricato"),
            outputs=gr.Textbox(),
            title="AI Image Detection - Errore"
        )
        error_interface.launch(share=True)

# 10. FUNZIONI AGGIUNTIVE PER KAGGLE
def quick_test():
    """
    Test rapido dell'interfaccia
    """
    if model is None:
        print("Modello non disponibile per il test")
        return
    
    print("Test dell'interfaccia...")
    detector = AIImageDetector(model, preprocess_val, tokenizer, device)
    
    # Test con immagine di esempio (sostituisci con un'immagine reale)
    try:
        # Crea un'immagine di test
        test_image = Image.new('RGB', (224, 224), color='red')
        result = detector.detect_single_image(test_image, detailed=True)
        print(f"Test completato: {result}")
        return result
    except Exception as e:
        print(f"Errore nel test: {e}")
        return None

# Test automatico all'avvio
if model is not None:
    print("\n🧪 Esecuzione test rapido...")
    test_result = quick_test()
    if test_result:
        print("✅ Test completato con successo!")
    else:
        print("⚠️ Test fallito, ma l'interfaccia dovrebbe comunque funzionare")

✓ gradio installato con successo
✓ open-clip-torch installato con successo
✓ timm installato con successo
✓ ftfy installato con successo
✓ regex installato con successo
✓ plotly installato con successo
✓ kaleido installato con successo
Dispositivo utilizzato: cuda
✓ Modello CLIP caricato con successo
=== AI IMAGE DETECTION CON GRADIO ===
✅ Modello caricato con successo!
🌐 Avvio interfaccia web...
🚀 Avvio dell'interfaccia AI Image Detection...
<gradio.components.markdown.Markdown object at 0x781ad88c9c90> <gradio.components.plot.Plot object at 0x781ad87f6890> <gradio.components.plot.Plot object at 0x781ad88e4650>
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://1291f1570c49f63c81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🧪 Esecuzione test rapido...
Test dell'interfaccia...
Test completato: {'is_ai_generated': True, 'most_likely_source': 'stable_diffusion', 'category_scores': {'real_photo': 0.05114610195159912, 'ai_generated': 0.05375513657927513, 'stable_diffusion': 0.0539827731748422, 'midjourney': 0.05184286584456762, 'dalle': 0.052672321597735085}, 'confidence': 0.0539827731748422, 'ai_probability': 0.9488538980484009, 'suspected_generator': 'stable_diffusion'}
✅ Test completato con successo!


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",